In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [66]:
def get_cols(df: pd.DataFrame, cols) -> pd.DataFrame:
    """
    Returns a dataframe with only the specified column(s).
    
    Parameters:
        df (DataFrame): The original dataframe.
        cols: The columns to include in the returned DataFrame.
    Returns:
        df (DataFrame): A subset with only the specified columns.
    """
    if (type(cols) is str):
        cols = [cols]
    elif (type(cols) is int):
        cols = [cols]
    if (type(cols) is not list):
        raise TypeError("Specified column(s) need to be of type str or list")
    return df[cols]

In [99]:
def get_rows_by_index(df: pd.DataFrame, rows) -> pd.DataFrame:
    """
    Returns a dataframe with only the specified row(s) by index.
    
    Parameters:
        df (DataFrame): The original dataframe.
        rows (list): The indices to include in the returned DataFrame.
    Returns:
        df (DataFrame): A subset with only the specified rows.
    """
    if (type(rows) is str):
        raise TypeError("Specified indices need to be of type int or list")
    if (type(rows) is int):
        return df.iloc[[rows]]
    elif (type(rows) is list):
        return df.iloc[rows]
        

In [ ]:
def get_rows_by_rule(df: pd.DataFrame, ) -> pd.DataFrame:
    

In [90]:
a = [1, 2, "ok"]
if (type(a) is int):
    print("list of strings")

In [ ]:
my_df = pd.DataFrame()
my_df["id"] = pd.Series([x for x in range(10)])
my_df["current_x"] = np.random.uniform(0, 1, 10)
my_df["current_y"] = np.random.uniform(0, 1, 10)
my_df["age"] = np.random.normal(45, 10, size = 10)
my_df["age"] = my_df["age"].astype(int)
my_df["state"] = pd.Series(0 for x in range(10))
print(my_df)
#print(get_cols(my_df, 2))
print(get_rows_by_index(my_df, [2,4]))

In [23]:
def csv_to_df(csv_path: str) -> pd.DataFrame:
    """
    Converts a csv to a dataframe and cleans it up.
    
    Parameters:
        csv_path (string): A system path to the csv.
    Returns:
        new_df (DataFrame): A dataframe.
    """
    if(Path(csv_path).is_file()):
        new_df = pd.read_csv(csv_path)
        return new_df
    else:
        print("File doesn't exist")

def clean_df(df_to_clean: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans a dataframe.
    
    Parameters:
        df_to_clean (DataFrame): A dataframe to clean.
    Returns:
        df (DataFrame): A subset with only the specified rows.
    """

csv_file = "C:/Users/Parthivnaresh/Documents/PracticeFile.csv"
running_df = csv_to_df(csv_file)
#clean_df = clean_def(running_df)
print(running_df)
#print(is_this_file)

    id      x      y   age  state
0    1  0.234  0.374    45    NaN
1    2  0.521  0.789    76    3.0
2    3  0.111  0.923    52    2.0
3    4  0.633  0.284    12    3.0
4    5  0.789  0.773     9    4.0
5    6  0.923  0.445    98    5.0
6    7  0.284  0.481    33    1.0
7    8  0.773  0.672    77    2.0
8    9  0.288  0.056    55    3.0
9   10  0.888  0.182  None    0.0
10  11  0.921  0.886    34    0.0
11  12  0.765  0.432    86    3.0
12  13  0.567  0.432    92    4.0
13  14  0.733    NaN     3    2.0
14  15  0.012  0.882    14    5.0
15  16  0.190  0.421    52    4.0
16  17  0.185  0.009    12    2.0
17  18  0.453  0.567    43    1.0
18  19  0.322  0.733    34    3.0
19  20  0.455  0.012    22    5.0
